In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 1. Data Preprocessing

In [12]:
df = pd.read_json("News_Category_Dataset_v3.json", lines=True)
df = pd.DataFrame(df)
categories = ["POLITICS", "TRAVEL", "SPORTS", "HOME & LIVING"]

df = df[df['category'].isin(categories)]

In [13]:
df= df.groupby('category').apply(lambda x: x.sample(1000, random_state=42)).reset_index(drop=True)

C:\Users\bbuser\AppData\Local\Temp\ipykernel_23372\3861801514.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df= df.groupby('category').apply(lambda x: x.sample(1000, random_state=42)).reset_index(drop=True)


In [14]:
df = df[['headline', 'category']]

In [15]:
df

,headline,category
0,"Busiest Shipping Day Of The Year Is Today, Ann...",HOME & LIVING
1,What To Watch On Netflix That’s New This Week ...,HOME & LIVING
2,Repurposing Idea Shows You How To Organize Hai...,HOME & LIVING
3,Company Buys $8000 Horse Lamp By Front Design ...,HOME & LIVING
4,Renovate for Rent,HOME & LIVING
...,...,...
3995,The 7 Most Mysterious Stone-Carved Faces That ...,TRAVEL
3996,Tips for a Stress-Free Family Summer Vacation,TRAVEL
3997,These Are The Busiest Flight Routes In The World,TRAVEL
3998,"This Is The Best, Most Underrated Travel Resource",TRAVEL


## 2. Vectorization

In [16]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['headline'])

## 3. Search Implementation

In [23]:
def search_query(query):
    vectorizing_query = vectorizer.transform([query])
    similarity = cosine_similarity(vectorizing_query, tfidf).flatten()
    top_result = similarity.argsort()[-10:][::-1]
    final_result = df.iloc[top_result]
    final_result['similarity_score'] = similarity[top_result]

    return final_result.sort_values(by='similarity_score', ascending=False)

## 4. User Experience

In [28]:
query = input("Enter a query to srach: ")
results = search_query(query)
print(results[['headline', 'category', 'similarity_score']])

                                              headline       category  \
1    What To Watch On Netflix That’s New This Week ...  HOME & LIVING   
105  What To Watch On Netflix That’s New This Week ...  HOME & LIVING   
909  What To Watch On Netflix That’s New This Week ...  HOME & LIVING   
138  What To Watch On Netflix That’s New This Week ...  HOME & LIVING   
373  What To Watch On Netflix That’s New This Week ...  HOME & LIVING   
851    The Best Shows To Watch On Netflix In July 2020  HOME & LIVING   
595  Here’s What Is Arriving On And Leaving Netflix...  HOME & LIVING   
337  The 10 Most Popular Shows On Netflix Right Now...  HOME & LIVING   
677  The 10 Most Popular Movies On Netflix Right No...  HOME & LIVING   
957  The Most Popular Shows On Netflix Right Now Be...  HOME & LIVING   

     similarity_score  
1            0.619403  
105          0.604480  
909          0.578482  
138          0.566096  
373          0.557950  
851          0.519476  
595          0.367667  
337 

C:\Users\bbuser\AppData\Local\Temp\ipykernel_23372\4144774753.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['similarity_score'] = similarity[top_result]
